In [1]:
from termcolor import colored
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
# other python utilities 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
from sklearn.metrics.pairwise import cosine_similarity
# for eval
from sklearn.model_selection import train_test_split
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.5f}'.format)


In [2]:
df = pd.read_csv("D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Oct_Forth_projectType.csv")

categories_cols = []

print(colored('categorizing the following columns for less memory usage...', 'green'))
for col in df.columns:
    #print(str(col))
    cats = len(df[col].unique())
    #print(cats)
    if cats<=10:
        categories_cols.append(col)
        print(col)
        
df[categories_cols] = df[categories_cols].astype('category')

categorizing the following columns for less memory usage...
Project Grade Level Category
Project Resource Category
Project Current Status
School Metro Type
School State
Donor State


In [10]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

print('# donations in Train set: ', f"{len(df_train):,}")
print('# donations in Test set: ',  f"{len(df_test):,}")

print('# Donors in Train set: ', f"{len(df_train['Donor ID'].unique()):,}")
print('# Donors in Test set: ',  f"{len(df_test['Donor ID'].unique()):,}")


print('# Donors in both Train and Test sets - the ones we choose for evaluation: ',
      colored(f"{len(df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())]['Donor ID'].unique()):,}", 'blue'))


df_train = df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())].reset_index(drop=True)
df_test = df_test[df_test['Donor ID'].isin(df_train['Donor ID'].values.tolist())].reset_index(drop=True)

# sum of donation in a grouped by donor id dataset
df_main_donor_index = df[df['Donor ID'].isin(df_test['Donor ID'])].groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_train_donor_index = df_train.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_test_donor_index = df_test.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')

# donations in Train set:  43,691
# donations in Test set:  4,855
# Donors in Train set:  795
# Donors in Test set:  618
# Donors in both Train and Test sets - the ones we choose for evaluation:  618


In [12]:
# aggregating projects and donors in donor_project
df = df[df['Donor ID'].isin(df_main_donor_index.index)].reset_index(drop=True)
print(f"len(df) = {len(df)}")

donor_project_df = df.groupby(['Donor ID', 'Project ID'])['Donation Amount'].sum().reset_index()
donors_projects_df_pivoted = donor_project_df.pivot('Donor ID', 'Project ID', 'Donation Amount').fillna(0)

donors_projects = donors_projects_df_pivoted.values
donors_id = donors_projects_df_pivoted.index
projects_id = donors_projects_df_pivoted.columns

print(f'len(donors_id): {len(donors_id)}')
print(f'len(projects_id): {len(projects_id)}')

len(df) = 46805
len(donors_id): 618
len(projects_id): 860


In [26]:
U, sigma, Vt = svds(donors_projects, k=50)
sigma = np.diag(sigma)
all_donor_predicted_preference = np.dot(np.dot(U, sigma), Vt)
svg_predicted_preference_df = pd.DataFrame(all_donor_predicted_preference, columns=projects_id, index=donors_id)#.transpose()

assert svg_predicted_preference_df.shape[1] == len(projects_id)
assert svg_predicted_preference_df.shape[0] == len(donors_id)


In [27]:
svg_predicted_preference_df.head(2)

Project ID,0026b34039dc4aba1fc028f25fe3deb3,0029e426fd3296af4fc333580fa895fe,004a152bbe8952ea5e9d5ef89c179933,0072118edd7e0c5e84d23be2424bebd9,008df5051d1a55dda66ce632084fae13,00be6e8f408fd6018383cde358d07106,00caf6fe22a68be347548718301a75e4,00ec6960b80c25e44ac0b5d24bea579d,010732a68a6a6a7a40b6827355bd2a04,013d10b67770d1742686168b8b070657,...,fb0043c9a5dc4a56db41909ff07bc54f,fb068b93ab567c7939a8d1ae556f5f8e,fb50a414956d2e6f888ce7feb0260953,fbce2a3f3700522e4e343794936e9742,fd2df103a3e79b222bbcf82278e07c0a,fdcdcf76362fc51d4be37feb5c10b6a7,fe2c82566b7837cf9f1030db73af3301,fe5d5922e8f78f94db77b5898814be34,fe6b6be5822cbd21a8dfdc9151eee3aa,fea476504f6dce5ab3a4de004bec5d0b
Donor ID,,,,,,,,,,,,,,,,,,,,,
009d5fc7b87883ffad248db5150bf1fc,-0.00043,0.05520,2.41597,-0.05567,0.00026,0.00412,0.59412,-0.05338,5.14275,0.02610,...,0.45000,-0.19801,-0.21497,0.06196,-0.00161,0.07545,0.04907,0.62175,-0.00041,-0.00070
011ca63acc31287a150a693d92235160,0.00009,0.14355,0.02051,0.00631,0.03469,0.03138,0.05352,0.01792,0.41434,-0.00555,...,0.22433,-0.00916,-0.03331,0.01142,-0.00126,0.04182,-0.01224,-0.03313,0.00008,0.00014


In [69]:
projects_df = df[['Project ID', 'Project Subject Subcategory Tree', 'Project Title', 'Project Need Statement', 'School State']].drop_duplicates(['Project ID']).reset_index(drop=True)

In [70]:
projects_df

,Project ID,Project Subject Subcategory Tree,Project Title,Project Need Statement,School State
0,0029e426fd3296af4fc333580fa895fe,"Character Education, Literature & Writing","Everyone Needs an Address, Especially Maniac M...",My students need a class set of the book Mania...,Georgia
1,00caf6fe22a68be347548718301a75e4,"Literacy, Special Needs",Culturally and Linguistically Diverse Learners...,"My students need a set of multicultural books,...",California
2,02180b9e9229c81e19f57b39b4dc14e8,"Community Service, Early Development",Dressing up for Pre-k,My students need new dramatic play clothes to ...,Texas
3,028f0150ded49457878902f065ae5f3a,"Literacy, Literature & Writing",My Fifth Graders Want to Read These Books!,"My students need graphic novels, books in a se...",California
4,02f2f8fc72b4714f5c6b98c04edb5913,Visual Arts,Help Us Give Back!,"My students need pottery tools, scrapers, and ...",California
...,...,...,...,...,...
855,5a7731a092f6f5d8cba47dfbc406ac1c,Special Needs,We're APPY to Learn with iPads!,My students need access to technology that is ...,Illinois
856,6818a99e012a9b76ae96f0657e4a346e,"Literature & Writing, Special Needs","""Flexibility At Its Best!""",My students need flexibility resources like ex...,Indiana
857,d12532af09030886e568be252fb297dd,"Health & Life Science, Mathematics",Chromebook Comets!!,My students need access to technology in order...,Pennsylvania
858,92ebef1d93b641699e8a7f84e681995a,"Literacy, Literature & Writing","Emerging Readers, Future Leaders!",My students need leveled classroom libraries f...,California


In [46]:
projects_df.duplicated(['Project ID']).sum()

0

In [76]:
class CFRecommender:
    

    """
Collaborate filtering using Singular Value Decomposition (SVD)
    """

    MODEL_NAME = 'Collaborative Filtering'


    def __init__(self, svg_predicted_preference_df: pd.DataFrame, projects_df: pd.DataFrame):

        self.svg_predicted_preference_df = svg_predicted_preference_df
        self.projects_df = projects_df



    def get_model_name(self):
        return self.MODEL_NAME



    def recommend_projects(self, donor_id: str, projects_to_ignore : list=[], top_n=10, df_test: pd.DataFrame = df_test):
        
        """
        
        """
        donated_projects_test = df_test_donor_index.loc[donor_id, 'Project ID']


        # if they have only one donation
        if type(donated_projects_test) == str:
            donated_projects_test  = [donated_projects_test]
        else:
            # other cases
            donated_projects_test  = list(donated_projects_test .values)




        # remove projects from ignore list if it is also in the test set
        wanted_in_test = []

        for project_id in projects_to_ignore:
            if project_id in donated_projects_test:
                wanted_in_test.append(project_id)
        
        if wanted_in_test != []: projects_to_ignore = set(projects_to_ignore).difference(wanted_in_test)


        # get recommendations
        donor_predictions = self.svg_predicted_preference_df.loc[donor_id, :].reset_index().rename(columns={donor_id: 'recommStrength'})
        

        # making sure we have all projects we want and sorting them
        recommendations_df = donor_predictions[~donor_predictions['Project ID'].isin(projects_to_ignore)].sort_values(by='recommStrength', ascending=False)[:top_n]


        # merging with projects dataset to get the descriptions
        recommendations_df_with_description = pd.merge(left = recommendations_df, right = projects_df, how='left', on='Project ID')#[['recommStrength', 'Project ID', 'Project Title', 'Project Need Statement']]
        

        return recommendations_df_with_description, projects_to_ignore


In [77]:
cf_model = CFRecommender(svg_predicted_preference_df, df)
donor_1 = "009d5fc7b87883ffad248db5150bf1fc"
recommendations_df, projects_to_ignore = cf_model.recommend_projects(donor_1, top_n=10)

In [78]:
recommendations_df

,Project ID,recommStrength,Project Subject Subcategory Tree,Project Title,Project Need Statement,School State
0,30f7fc4700fb7523586fd9de798302e6,6.50794,"Music, Performing Arts",Standing Up for Our Band - Part 2,My students need new music stands for our Big ...,Texas
1,010732a68a6a6a7a40b6827355bd2a04,5.14275,"Literacy, Performing Arts",Help Little Voices Be Heard! Mics For Our Show...,My students need wireless microphones for dram...,Texas
2,d08fa4c49e879d46d01867817a9f5e7d,5.10322,"Performing Arts, Special Needs",Dance Create Elevate!,My students need costumes to present the work ...,New York
3,d2926b1c30b4ccbaf4a104c7430ce10a,3.83695,Music,I Want To Bang On the Drum All Day!,My students need 2 drums to start building our...,Indiana
4,b82b4bc5891d416a23a401f1b054af4d,3.65453,"Gym & Fitness, Health & Wellness","Fitness on Your Mark, Get Set, Go!","My students need a portable PA system, balls, ...",New York
5,f83a589eb471f3ff71e27c4d44f3e45d,3.47493,"Early Development, Social Sciences",We Need A Robot Like Nao!,"My students need a Nao, an autonomous, program...",California
6,6e6e92b7478fe0fc6a1a1a69bb8280c9,3.47095,"Literacy, Literature & Writing",For the Love of Pete!,My students need their own copies of 5 differe...,Texas
7,bea19546b0c8230223353474c6eb8335,3.34196,Literacy,iStation Learning Part 1,My students need an ipad mini to use during In...,Texas
8,d3831ea2611d92ca62a91a55454085ce,3.31125,Literacy,Books of Our Very Own!,"My students need books of their very own, such...",Texas
9,2317d14aae9d3f4d92f5da30a62ff3a4,3.06392,"Literacy, Mathematics",Let's Get Down and Boogie!,My students need 12 boogie boards with protect...,Florida


In [79]:
def get_projects_donated(donor_id: str, df_donor_indexed: pd.DataFrame) -> set:

    """ 
    get the project one has donated to in a specific df
    """
    try:
        donated_projects = df_donor_indexed.loc[donor_id]['Project ID']

        return set(donated_projects if type(donated_projects) == pd.Series else [donated_projects])
        
    except KeyError:
        return []

In [82]:
#Top-N accuracy metrics 
EVAL_RANDOM_SAMPLE_NON_INTERACTED_PROJECTS = 100


class ModelEvaluator:

    def __init__(self, df_main_donor_index, projects_id):

        self.df_main_donor_index = df_main_donor_index
        self.df_test_donor_index = df_test_donor_index
        self.df_train_donor_index = df_train_donor_index
        self.projects_id = projects_id
        


    def get_not_donated_projects_sample(self, donor_id: str, sample_size: int, seed=42) -> set:
        
        """
        input: donor_id
        output: a set of not donated projects in df
        """
        
        donated_projects = get_projects_donated(donor_id=donor_id, df_donor_indexed=self.df_main_donor_index)

        not_donated_projects = [x for x in self.projects_id if x not in donated_projects]
        not_donated_projects_sample = random.sample(not_donated_projects, sample_size)
        
        return set(not_donated_projects_sample)



    def _verify_hit_top_n(self, project_id: str, recommended_projects: pd.Series, top_n) -> (bool, int):
        """ 
        input: one project id (a project our donor has donated to), a set of recommended projects
        output: the index of that project_id among all the recomms
        """
        try:
            index = next(i for i, c in enumerate(recommended_projects) if c == project_id)
        except:
            index = -1

        hit = int(index in range(0, top_n))

        return hit, index
        

    def evaluate_model_for_donor(self, model, donor_id: str):
        """
        evaluates the recommendations recommended to one donor

        """

        # what donor has donated to in the test set
        if type(self.df_test_donor_index.loc[donor_id, 'Project ID']) == pd.Series:
            donated_projects_test = set(self.df_test_donor_index.loc[donor_id, 'Project ID'])
        else:
            donated_projects_test = set([self.df_test_donor_index.loc[donor_id, 'Project ID']])

        donated_projects_count_test = len(donated_projects_test)

        # overlap
            
        df_train_d = df_train_donor_index.loc[donor_id, 'Project ID']
        df_test_d = df_test_donor_index.loc[donor_id, 'Project ID']

        if type(df_test_d) == str: 
            df_test_d = [df_test_d]
        else: df_test_d = list(df_test_d.values)


        if type(df_train_d) == str: 
            df_train_d = [df_train_d]
        else: df_train_d = list(df_train_d.values)

        overlap = 0
        for project_id in df_test_d:
            if project_id in df_train_d:
                overlap+=1
            else: continue


        recommendations_df, projects_to_ignore = model.recommend_projects(donor_id, projects_to_ignore= get_projects_donated(donor_id, 
        df_donor_indexed = df_train_donor_index), top_n = None, df_test=df_test)

        # if correct we have ranked all the projects except the ones the donor have been only in training set of 
        # this specific donor
        assert len(recommendations_df) == len(projects_id) - len(projects_to_ignore)


        
        hits_at_3_count = 0
        hits_at_5_count = 0
        hits_at_10_count = 0

         
          
        for project_id in donated_projects_test:


            # first get a sample of the ones he\she has not donated to 
            not_donated_projects_sample =  self.get_not_donated_projects_sample(donor_id, sample_size = EVAL_RANDOM_SAMPLE_NON_INTERACTED_PROJECTS, seed = 42)
                    
                    
            # add a donated project to a list of 100 projects this donor has not interacted with
            validation_projects = not_donated_projects_sample.union(set([project_id]))


            # if true means we have correctly identified the projects they have not interacted with
            assert len(not_donated_projects_sample)+1 == len(validation_projects)





            recommendations_df_ =  recommendations_df[recommendations_df['Project ID'].isin(validation_projects)].reset_index(drop=True)
 

            recommended_project_ids = recommendations_df_['Project ID'].values
            assert len(recommended_project_ids) == 101 , print(len(recommended_project_ids))


            hit_at_3, index_at_3 = self._verify_hit_top_n(project_id, recommended_project_ids, 3)
            hits_at_3_count += hit_at_3

            hit_at_5, index_at_5 = self._verify_hit_top_n(project_id, recommended_project_ids, 5)
            hits_at_5_count += hit_at_5

            hit_at_10, index_at_10 = self._verify_hit_top_n(project_id, recommended_project_ids, 10)
            hits_at_10_count += hit_at_10



        # ---------------------------- Recall --------------------------#
        recall_at_3 = hits_at_3_count/float(donated_projects_count_test)
        recall_at_5 = hits_at_5_count / float(donated_projects_count_test)
        recall_at_10 = hits_at_10_count / float(donated_projects_count_test)


        donor_metrics = {'donor_id': donor_id,
                        'hits@3_count':hits_at_3_count, 
                         'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'donated_count': donated_projects_count_test,
                          'project_overlap_test_train_count': overlap,
                          'recall@3': recall_at_3,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return donor_metrics

    def evaluate_model(self, model):
        """
        aggregates the results of evaluate_model_for_donor
        """
        metrics = []

        for idx, donor_id in enumerate(list(self.df_test_donor_index.index.unique().values)):
            
            donor_metrics = self.evaluate_model_for_donor(model, donor_id)
            if idx%500 ==0: print('%d donors processed' % idx)

            metrics.append(donor_metrics)

            detailed_results_df = pd.DataFrame(metrics).sort_values('donated_count',  ascending=False).reset_index(drop=True)
            
        glob_num_donations = float(detailed_results_df['donated_count'].sum())

        global_recall_at_3 = detailed_results_df['hits@3_count'].sum()/ glob_num_donations
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum()/ glob_num_donations
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum()/ glob_num_donations

        global_metrics = {'modelName': model.get_model_name(),
                          'recall@3': global_recall_at_3,
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    




In [86]:
print(colored('SVG: ', 'green'))

model_evaluator = ModelEvaluator(df_main_donor_index, projects_id)
global_metrics, detailed_results_df = model_evaluator.evaluate_model(cf_model)


print('\nGlobal metrics:\n%s' % global_metrics)
detailed_results_df = detailed_results_df[['donor_id', 'donated_count', "hits@3_count", 'hits@5_count', 'hits@10_count', 'recall@3','recall@5','recall@10']]
detailed_results_df.head(10)

SVG: 
0 donors processed
500 donors processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@3': 0.6124537958487347, 'recall@5': 0.6994597668467444, 'recall@10': 0.809212396929201}


,donor_id,donated_count,hits@3_count,hits@5_count,hits@10_count,recall@3,recall@5,recall@10
0,237db43817f34988f9d543ca518be4ee,90,90,90,90,1.00000,1.00000,1.00000
1,b51c76411b51751f45527c63c69ead9e,67,66,67,67,0.98507,1.00000,1.00000
2,4416745560343f14a74dedcda4ec03b0,60,58,59,59,0.96667,0.98333,0.98333
3,39df9399f5384334a42905bcf0acdcbf,57,57,57,57,1.00000,1.00000,1.00000
4,c376c98b0cdb746cf025bb21ee810376,49,34,40,44,0.69388,0.81633,0.89796
5,a299db9679f7746a805fbc300362191d,44,20,27,34,0.45455,0.61364,0.77273
6,98c4cd327c417683cd76a2ac19fc6254,43,39,39,42,0.90698,0.90698,0.97674
7,03fa60275eb66e873c30ecb86840df4b,42,42,42,42,1.00000,1.00000,1.00000
8,24ecca49933c30a0beb83090591720c0,41,41,41,41,1.00000,1.00000,1.00000
9,609e28d99b36d35679ae56268e4dddc3,39,27,29,34,0.69231,0.74359,0.87179
